In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

#!ls "/content/drive/My Drive/processed_images"

Mounted at /content/drive


In [ ]:
import torch
import numpy as np
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import Dataset, random_split
from PIL import Image
import os
import pandas as pd

transform = transforms.Compose([
    transforms.RandomRotation(30),      # rotate +/- 30 degrees
    transforms.RandomHorizontalFlip(),  # rHorizontally flip the given image randomly with a given probability (default p=0.5)
    #transforms.RandomVerticalFlip() #Vertically flip the given image randomly with a given probability (default p=0.5), not recommended for medical images
    transforms.Resize((224, 224)),       #  be sure to pass in a list or a tuple
    transforms.CenterCrop(224),         # crop longest side to 224 pixels at center
    transforms.RandomAdjustSharpness(1.5, p=0.5), #
    transforms.RandomAdjustSharpness(0.5, p=0.5),
    transforms.RandomAutocontrast(p=0.5),
    transforms.RandomEqualize(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                          [0.229, 0.224, 0.225])
])

class CovidSeverityDataset(Dataset):
    def __init__(self, root_dir, transform = transform, split_lengths = [0.7, 0.1, 0.2], split_seed = 42, batch_size = 10, shuffle = True, num_workers = [2,0,0]):
      self.root_dir = root_dir
      self.csv_path = self.root_dir + "data_processing/combined_cxr_metadata.csv"
      self.data_path = self.root_dir 
      self.transform = transform
      self.split_lengths = split_lengths
      self.split_seed = split_seed
      self.batch_size = batch_size
      self.shuffle = shuffle
      self.num_workers = num_workers
      self.dataframe = pd.read_csv(self.csv_path, index_col=0)
  
    def __len__(self):
      return len(self.dataframe)

    def __getitem__(self, index):
      img_path = os.path.join(self.data_path, self.dataframe.iloc[index, 0])
      image = Image.open(img_path)
      y_label = torch.round(torch.tensor(self.dataframe.iloc[index, 1]))
      if self.transform:
        image = self.transform(image)
      else:
        convert_tensor = transforms.PILToTensor()
        image = convert_tensor(image)
      return (image.float(), y_label.to(torch.int64))
  
    def get_subsets(self):
      subsets = random_split(self, self.split_lengths, generator=torch.Generator().manual_seed(self.split_seed))
      train = torch.utils.data.DataLoader(dataset=subsets[0], batch_size=self.batch_size, shuffle=self.shuffle, num_workers=self.num_workers[0])
      val = torch.utils.data.DataLoader(dataset=subsets[1], batch_size=self.batch_size, shuffle=self.shuffle, num_workers=self.num_workers[1])
      test = torch.utils.data.DataLoader(dataset=subsets[2], batch_size=self.batch_size, shuffle=self.shuffle, num_workers=self.num_workers[2])
      return train, val, test


# How to use this class?
# root_dir = "/content/drive/MyDrive/Mila/Winter_2023/ift6759_project/"
# dataset = CovidSeverityDataset(root_dir, transform = False)
# train, val, test = dataset.get_subsets()


#!ls "/content/drive/My Drive/processed_images"

In [ ]:
root_dir = "/content/drive/My Drive/processed_images/"
dataset = CovidSeverityDataset(root_dir, transform = False)
train, val, test = dataset.get_subsets()

# Evaluating VGNETT out of box

In [ ]:
print(len(test))
# evaluation dataset : 447/1440 -> 0.31 

144


In [ ]:
import torchvision.models as models
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()  

In [ ]:
from torchvision import models
vgg16 = models.vgg16_bn(pretrained=True)
print(vgg16.classifier[6].out_features) # 1000 

for param in vgg16.features.parameters():
    param.require_grad = False

# Newly created modules have require_grad=True by default
num_features = vgg16.classifier[6].in_features
print(num_features)
features = list(vgg16.classifier.children())[:-1] # Remove last layer
features.extend([nn.Linear(num_features, 7)]) # Add our layer with 4 outputs
vgg16.classifier = nn.Sequential(*features) # Replace the model classifier
print(vgg16)

1000
4096
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2

In [ ]:
def eval_model(vgg, criterion):
    since = time.time()
    avg_loss = 0
    avg_acc = 0
    loss_test = 0
    acc_test = 0
    
    test_batches = len(test)
    print("Evaluating model")
    print('-' * 10)
    
    for i, data in enumerate(test):
        #print(i)
        if i % 100 == 0:
            print("\rTest batch {}/{}".format(i, test_batches), end='', flush=True)

        vgg.train(False)
        vgg.eval()
        inputs, labels = data
        #print(inputs)
        labels = labels.type(torch.LongTensor)
        #if use_gpu:
        inputs, labels = Variable(inputs.cuda(), volatile=True), Variable(labels.cuda(), volatile=True)
        #else:
        #    inputs, labels = Variable(inputs, volatile=True), Variable(labels, volatile=True)

        outputs = vgg(inputs)

        _, preds = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)
        print(loss)
        #loss_test += loss.data[0]
        acc_test += torch.sum(preds == labels.data)
        print(acc_test)
        print(len(labels))

        del inputs, labels, outputs, preds
        torch.cuda.empty_cache()
        
    #avg_loss = loss_test / dataset_sizes[TEST]
    #avg_acc = acc_test / dataset_sizes[TEST]
    
    elapsed_time = time.time() - since
    print()
    print("Evaluation completed in {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
    #print("Avg loss (test): {:.4f}".format(avg_loss))
    #print("Avg acc (test): {:.4f}".format(avg_acc))
    print('-' * 10)

In [ ]:
#if use_gpu:

vgg16.cuda() #.cuda() will move everything to the GPU side
    
criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
print("Test before training")
eval_model(vgg16, criterion)

Test before training
Evaluating model
----------
tensor(2., dtype=torch.float64)
tensor(2., dtype=torch.float64)
tensor(5., dtype=torch.float64)
tensor(2., dtype=torch.float64)
tensor(3., dtype=torch.float64)
tensor(4., dtype=torch.float64)
tensor(3., dtype=torch.float64)
tensor(4., dtype=torch.float64)
tensor(3., dtype=torch.float64)
tensor(2., dtype=torch.float64)
0
Test batch 0/144tensor(1.8794, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3, device='cuda:0')
10


<ipython-input-43-3efadbe1a2f3>:23: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  inputs, labels = Variable(inputs.cuda(), volatile=True), Variable(labels.cuda(), volatile=True)


tensor(3., dtype=torch.float64)
tensor(2., dtype=torch.float64)
tensor(1., dtype=torch.float64)
tensor(1., dtype=torch.float64)
tensor(3., dtype=torch.float64)
tensor(3., dtype=torch.float64)
tensor(3., dtype=torch.float64)
tensor(4., dtype=torch.float64)
tensor(3., dtype=torch.float64)
tensor(3., dtype=torch.float64)
1
tensor(2.3361, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(9, device='cuda:0')
10
tensor(1., dtype=torch.float64)
tensor(3., dtype=torch.float64)
tensor(4., dtype=torch.float64)
tensor(2., dtype=torch.float64)
tensor(3., dtype=torch.float64)
tensor(2., dtype=torch.float64)
tensor(4., dtype=torch.float64)
tensor(4., dtype=torch.float64)
tensor(4., dtype=torch.float64)
tensor(5., dtype=torch.float64)
2
tensor(2.4214, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(11, device='cuda:0')
10
tensor(4., dtype=torch.float64)
tensor(4., dtype=torch.float64)
tensor(5., dtype=torch.float64)
tensor(3., dtype=torch.float64)
tensor(3., dtype=torch.float64)
tensor(1., dtyp

In [ ]:
def train_model(vgg, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()
    best_model_wts = copy.deepcopy(vgg.state_dict())
    best_acc = 0.0
    
    avg_loss = 0
    avg_acc = 0
    avg_loss_val = 0
    avg_acc_val = 0
    
    train_batches = len(train)
    val_batches = len(train)
    
    for epoch in range(num_epochs):
        print("Epoch {}/{}".format(epoch, num_epochs))
        print('-' * 10)
        
        loss_train = 0
        loss_val = 0
        acc_train = 0
        acc_val = 0
        
        vgg.train(True)
        count_of_train_data = 0
        for i, data in enumerate(train):
            count_of_train_data = count_of_train_data + len(train)
            if i % 100 == 0:
                print("\rTraining batch {}/{}".format(i, train_batches / 2), end='', flush=True)
                
            # Use half training dataset
            if i >= train_batches / 2:
                break
                
            inputs, labels = data
            labels = labels.type(torch.LongTensor)
            
            #if use_gpu:
            inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
            #else:
            #    inputs, labels = Variable(inputs), Variable(labels)
            
            optimizer.zero_grad()
            
            outputs = vgg(inputs)
            
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            #print(loss)
            #loss_train += loss.data[0]
            acc_train += torch.sum(preds == labels.data)
            
            del inputs, labels, outputs, preds
            torch.cuda.empty_cache()
            #break
        
        print()
        # * 2 as we only used half of the dataset
        avg_loss = loss_train / count_of_train_data
        avg_acc = acc_train / count_of_train_data
        
        vgg.train(False)
        vgg.eval()
            
        count_of_data = 0 
        for i, data in enumerate(val):
            count_of_data = count_of_data + len(val)
            if i % 100 == 0:
                print("\rValidation batch {}/{}".format(i, val_batches), end='', flush=True)
                
            inputs, labels = data
            labels = labels.type(torch.LongTensor)
            #if use_gpu:
            inputs, labels = Variable(inputs.cuda(), volatile=True), Variable(labels.cuda(), volatile=True)
            #else:
            #    inputs, labels = Variable(inputs, volatile=True), Variable(labels, volatile=True)
            
            optimizer.zero_grad()
            
            outputs = vgg(inputs)
            
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            
            #loss_val += loss.data[0]
            acc_val += torch.sum(preds == labels.data)
            
            del inputs, labels, outputs, preds
            torch.cuda.empty_cache()
            #break
            
        
        
        avg_loss_val = loss_val / count_of_data
        avg_acc_val = acc_val / count_of_data
        
        print()
        print("Epoch {} result: ".format(epoch))
        print("Avg loss (train): {:.4f}".format(avg_loss))
        print("Avg acc (train): {:.4f}".format(avg_acc))
        print("Avg loss (val): {:.4f}".format(avg_loss_val))
        print("Avg acc (val): {:.4f}".format(avg_acc_val))
        print('-' * 10)
        print()
        
        if avg_acc_val > best_acc:
            best_acc = avg_acc_val
            best_model_wts = copy.deepcopy(vgg.state_dict())
        
    elapsed_time = time.time() - since
    print()
    print("Training completed in {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
    print("Best acc: {:.4f}".format(best_acc))
    
    vgg.load_state_dict(best_model_wts)
    return vgg

In [ ]:
vgg16 = train_model(vgg16, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=30)
torch.save(vgg16.state_dict(), 'VGG16_v2-OCT_Retina_half_dataset.pt')

Epoch 0/30
----------
Training batch 200/251.0
Validation batch 0/502

<ipython-input-14-3e252b45544f>:76: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  inputs, labels = Variable(inputs.cuda(), volatile=True), Variable(labels.cuda(), volatile=True)



Epoch 0 result: 
Avg loss (train): 0.0000
Avg acc (train): 0.0067
Avg loss (val): 0.0000
Avg acc (val): 0.0540
----------

Epoch 1/30
----------
Training batch 100/251.0

FileNotFoundError: ignored